# 1. Importar las bibliotecas a utilizar:


In [ ]:
import escher
%matplotlib inline
import cobra  
from cobra.io import load_json_model
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
from IPython.display import display
import gurobipy
from cameo.strain_design.heuristic.evolutionary_based import OptGene
from cameo.visualization.plotting.with_plotly import PlotlyPlotter
plotter = PlotlyPlotter()

# 2. Cargar Modelos

In [ ]:
modelCoreA = cobra.io.load_json_model("./models/modelCoreA_bound.json")
modelCore = cobra.io.load_json_model("./models/modelCore_bound.json")
modeliML = cobra.io.load_json_model("./models/modeliML_bound.json")
modeliMLA = cobra.io.load_json_model("./models/modeliMLA_bound.json")

# 3. Metodos de Optimización

Para utilizar OptGene se deben definir el objeto de OptGene con el modelo respectivo:

In [ ]:
optgeneCore = OptGene(modelCore)
optgeneCoreA = OptGene(modelCoreA)
optgeneiML = OptGene(modeliML)
optgeneiMLA = OptGene(modeliMLA)

Creo una función que evalua OptGene con diferentes parámetros en cuanto a número de evaluaciones y máxima cantidad de knockouts de reacciones: 

In [ ]:
def OptGeneIterations(model):
    print('modelo:', model)
    maxEval = [10,100,1000,10000,20000]
    maxKO = [1,2,3,4,5,6]
    listResults = []
    for ko in maxKO:
        for ev in maxEval:
            if(model == 'core'):
                
                resultCore = optgeneCore.run(target="EX_btd_RR_e", biomass="BIOMASS_Ecoli_core_w_GAM",
                              substrate="glc__D_e",
                              max_evaluations=ev, max_knockouts=ko,
                              plot=False, growth_coupled=True)
                
                listResults.append(resultCore)
            if(model == 'coreA'):
                resultCoreA = optgeneCoreA.run(target="EX_btd_RR_e",
                              biomass="BIOMASS_Ecoli_core_w_GAM",
                              substrate="glc__D_e",
                              max_evaluations=ev, max_knockouts=ko,
                              plot=False, growth_coupled=True)
                listResults.append(resultCoreA)
                
            if(model == 'iML'):
                resultiML = optgeneiML.run(target="EX_btd_RR_e", 
                             biomass="BIOMASS_Ec_iML1515_core_75p37M",
                             substrate="glc__D_e",
                             max_evaluations=ev, max_knockouts=ko,
                             plot=False, growth_coupled=True)
                listResults.append(resultiML)
                
            if(model == 'iMLA'):
                resultiMLA = optgeneiMLA.run(target="EX_btd_RR_e", 
                             biomass="BIOMASS_Ec_iML1515_core_75p37M",
                             substrate="glc__D_e",
                             max_evaluations=ev, max_knockouts=ko,
                             plot=False, growth_coupled=True)
                listResults.append(resultiMLA)
    return listResults

Defino una función que toma los resultados y lo transforma en una matriz para mejor devolución de los resultados:

In [ ]:
def toMat(listResult):
    x = np.array(listResult)
    shape = (6,5)
    y = x.reshape(shape)
    return y


Defino una función que busca el máximo valor de BPCY para saber cual fue el mejor modelo optimizado:

In [ ]:
def maxFitness(mat):
    maxVal = 0
    row = 0
    col = 0
    for i in range(6):
        for x in range(5):
            fitness = mat[i][x].data_frame['fitness']
            if len(fitness) > 0:
                maxfit = max(fitness)
                if maxfit>maxVal:
                    maxVal = maxfit
                    row = i
                    col = x
    return row, col, maxVal

In [ ]:
#### primero se crean las listas
listCore = OptGeneIterations('core')
listCoreA = OptGeneIterations('coreA')
listiML = OptGeneIterations('iML')
listiMLA = OptGeneIterations('iMLA')

### Se pasan a matriz
matCore = toMat(listCore)
matCoreA= toMat(listCoreA)
matiML = toMat(listiML)
matiMLA = toMat(listiMLA)

rowC, colC, maxValC = maxFitness(matCore)
rowCA, colCA, maxValCA = maxFitness(matCoreA)
rowI, colI, maxValI = maxFitness(matiML)
rowIA, colIA, maxValIA = maxFitness(matiMLA)